In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#dados que serão utilizados
url = "https://mantis.saojudas.base2.com.br/login_page.php"

resumo = ["resumo tarefa"]
descricao = ["descricao tarefa"]

In [2]:
#função para realizar o login
def login(navegador):
    #localiza e digita nos campos de usuario e senha
    navegador.find_element_by_xpath('//*[@id="username"]').send_keys('Fernanda')
    navegador.find_element_by_xpath('//*[@id="login-form"]/fieldset/input[2]').click()
    navegador.find_element_by_xpath('//*[@id="password"]').send_keys(81721367)
    navegador.find_element_by_xpath('//*[@id="login-form"]/fieldset/input[3]').click()
    assert "https://mantis.saojudas.base2.com.br/my_view_page.php" in navegador.current_url
    
    #acessa página de reportar bug
    navegador.find_element_by_xpath('//*[@id="navbar-container"]/div[2]/ul/li[1]/div/a').click()    
    assert "https://mantis.saojudas.base2.com.br/bug_report_page.php" in navegador.current_url

In [3]:
#Função para criar preenchendo apenas os campos requeridos
def criar_tarefa_preenchendo_apenas_campos_requeridos():
    #abre o navegador e acessa a url
    navegador = webdriver.Chrome()
    navegador.get(url)
    
    #realiza o login
    login(navegador)
    
    #localiza e digita nos campos requeridos de resumo e descricao
    navegador.find_element_by_name('summary').send_keys(resumo[0])
    navegador.find_element_by_name('description').send_keys(descricao[0])
    navegador.find_element_by_xpath('//*[@id="report_bug_form"]/div/div[2]/div[2]/input').click()
    
    #verifica se a tarefa foi criada
    assert "Operação realizada com sucesso." in navegador.page_source
    
    #verifica se o sistema retornou os detalhes da tarefa criada
    try:
        WebDriverWait(navegador, 10).until(
            EC.url_contains("https://mantis.saojudas.base2.com.br/view.php")        
        )
    finally:
        navegador.close()
        print("Criar tarefa preenchendo apenas campos requeridos - OK")
    
criar_tarefa_preenchendo_apenas_campos_requeridos()

Criar tarefa preenchendo apenas campos requeridos - OK


In [4]:
#Função para criar tarefa preenchendo apenas campos input
def criar_tarefa_preenchendo_input():
    #abre o navegador e acessa a url
    navegador = webdriver.Chrome()
    navegador.get(url)
    
    #realiza o login
    login(navegador)

    #localiza e digita nos campos de input resumo, descricao, perfil, passos, 
    # informacao adicional e marcadores
    navegador.find_element_by_name('summary').send_keys(resumo[0])
    navegador.find_element_by_name('description').send_keys(descricao[0])
    
    try:
        if navegador.find_element_by_css_selector('#profile_closed_link').is_displayed():
            navegador.find_element_by_css_selector('#profile_closed_link').click()
        WebDriverWait(navegador, 10).until(
            EC.visibility_of(navegador.find_element_by_xpath('//*[@id="profile_open"]/table'))
        )
    finally:
        navegador.find_element_by_name('platform').send_keys("plataforma tarefa")
        navegador.find_element_by_name('os').send_keys("so tarefa")
        navegador.find_element_by_name('os_build').send_keys("versao so")
        navegador.find_element_by_name('steps_to_reproduce').send_keys("passos tarefa")
        navegador.find_element_by_name('additional_info').send_keys("informacoes tarefa")
        navegador.find_element_by_name('tag_string').send_keys('tag')
        navegador.find_element_by_xpath('//*[@id="report_bug_form"]/div/div[2]/div[2]/input').click()

    #verifica se a tarefa foi criada
    assert "Operação realizada com sucesso." in navegador.page_source
    
    #verifica se o sistema retornou os detalhes da tarefa criada
    try:
        WebDriverWait(navegador, 10).until(
            EC.url_contains("https://mantis.saojudas.base2.com.br/view.php")        
        )
    finally:
        navegador.close()
        print("Criar tarefa preenchendo campos input - OK")
    
    
criar_tarefa_preenchendo_input()

Criar tarefa preenchendo campos input - OK


In [5]:
#Função para criar uma tarefa sem preencher campos
def criar_tarefa_sem_preencher_nenhum_campo():
    #abre o navegador e acessa a url
    navegador = webdriver.Chrome()
    navegador.get(url)
    
    #realiza o login
    login(navegador)

    #localiza e clica em criar tarefa
    navegador.find_element_by_xpath('//*[@id="report_bug_form"]/div/div[2]/div[2]/input').click()
    
    #verifica se o navegador mudou de url
    assert "https://mantis.saojudas.base2.com.br/bug_report_page.php" in navegador.current_url
    
    navegador.close()  
    print("Criar tarefa sem preencher nenhum campo - OK")
    
criar_tarefa_sem_preencher_nenhum_campo()

Criar tarefa sem preencher nenhum campo - OK


In [6]:
## código auxiliar para carregar arquivos

from selenium.webdriver.remote.webelement import WebElement
import os.path

# JavaScript: HTML5 File drop
# source            : https://gist.github.com/florentbr/0eff8b785e85e93ecc3ce500169bd676
# param1 WebElement : Drop area element
# param2 Double     : Optional - Drop offset x relative to the top/left corner of the drop area. Center if 0.
# param3 Double     : Optional - Drop offset y relative to the top/left corner of the drop area. Center if 0.
# return WebElement : File input
JS_DROP_FILES = "var c=arguments,b=c[0],k=c[1];c=c[2];for(var d=b.ownerDocument||document,l=0;;){var e=b.getBoundingClientRect(),g=e.left+(k||e.width/2),h=e.top+(c||e.height/2),f=d.elementFromPoint(g,h);if(f&&b.contains(f))break;if(1<++l)throw b=Error('Element not interactable'),b.code=15,b;b.scrollIntoView({behavior:'instant',block:'center',inline:'center'})}var a=d.createElement('INPUT');a.setAttribute('type','file');a.setAttribute('multiple','');a.setAttribute('style','position:fixed;z-index:2147483647;left:0;top:0;');a.onchange=function(b){a.parentElement.removeChild(a);b.stopPropagation();var c={constructor:DataTransfer,effectAllowed:'all',dropEffect:'none',types:['Files'],files:a.files,setData:function(){},getData:function(){},clearData:function(){},setDragImage:function(){}};window.DataTransferItemList&&(c.items=Object.setPrototypeOf(Array.prototype.map.call(a.files,function(a){return{constructor:DataTransferItem,kind:'file',type:a.type,getAsFile:function(){return a},getAsString:function(b){var c=new FileReader;c.onload=function(a){b(a.target.result)};c.readAsText(a)}}}),{constructor:DataTransferItemList,add:function(){},clear:function(){},remove:function(){}}));['dragenter','dragover','drop'].forEach(function(a){var b=d.createEvent('DragEvent');b.initMouseEvent(a,!0,!0,d.defaultView,0,0,0,g,h,!1,!1,!1,!1,0,null);Object.setPrototypeOf(b,null);b.dataTransfer=c;Object.setPrototypeOf(b,DragEvent.prototype);f.dispatchEvent(b)})};d.documentElement.appendChild(a);a.getBoundingClientRect();return a;"

def drop_files(element, files, offsetX=0, offsetY=0):
    driver = element.parent
    isLocal = not driver._is_remote or '127.0.0.1' in driver.command_executor._url
    paths = []
    
    # ensure files are present, and upload to the remote server if session is remote
    for file in (files if isinstance(files, list) else [files]) :
        if not os.path.isfile(file) :
            raise FileNotFoundError(file)
        paths.append(file if isLocal else element._upload(file))
    
    value = '\n'.join(paths)
    elm_input = driver.execute_script(JS_DROP_FILES, element, offsetX, offsetY)
    elm_input._execute('sendKeysToElement', {'value': [value], 'text': value})

WebElement.drop_files = drop_files

In [9]:
#Função para criar tarefa anexando um arquivo valido
def anexar_arquivo_valido():    
    #abre navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)

    #realiza login
    login(navegador)

    #localiza e preenche campos requeridos
    navegador.find_element_by_name('summary').send_keys(resumo[0])
    navegador.find_element_by_name('description').send_keys(descricao[0])
    
    #carrega arquivo teste.pdf
    navegador.find_element_by_class_name('dropzone').drop_files(r'C:\Users\Fernanda\Desktop\base2\teste.pdf') ### MUDAR
    navegador.find_element_by_xpath('//*[@id="report_bug_form"]/div/div[2]/div[2]/input').click()
    
    #verifica se o sistema retorna os detalhes da tarefa criada
    try:
        WebDriverWait(navegador, 10).until(
            EC.url_contains("https://mantis.saojudas.base2.com.br/view.php")        
        )
    finally:
        navegador.close()
        print("Anexar arquivo válido - OK")
    
anexar_arquivo_valido()

Anexar arquivo válido - OK


In [16]:
#Função para criar tarefa anexando arquivo inválido
def arquivo_excedendo_tamanho_maximo():  
    #abre o navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)

    #realiza login
    login(navegador)

    #carrega arquivo teste_excedido.csv
    navegador.find_element_by_class_name('dropzone').drop_files(r'C:\Users\Fernanda\Desktop\base2\teste_excedido.csv') ### MUDAR
    
    #verifica se o sistema retorna mensagem de alerta
    try:
        alert = WebDriverWait(navegador, 10).until(EC.alert_is_present())
        assert "tamanho máximo de arquivo permitido" in alert.text
        alert.accept()
    finally:
        navegador.close()
        print("Anexar arquivo excedendo tamanho máximo - OK")
    
arquivo_excedendo_tamanho_maximo()

Anexar arquivo excedendo tamanho máximo - OK


In [10]:
#Função para criar tarefas em sequencia
def criar_tarefas_em_sequencia():
    #abre navegador e acessa url
    navegador = webdriver.Chrome()
    navegador.get(url)
    
    #realiza login
    login(navegador)

    #localiza e preenche campos requeridos
    navegador.find_element_by_name('summary').send_keys(resumo[0])
    navegador.find_element_by_name('description').send_keys(descricao[0])
    
    #localiza e seleciona checkbox para continuar criando tarefas
    navegador.find_element_by_xpath(".//*[contains(text(), 'selecione para criar mais tarefas')]").click()
    navegador.find_element_by_xpath('//*[@id="report_bug_form"]/div/div[2]/div[2]/input').click()

    #verifica se o sistema retorna para a página de criar tarefas
    try:
        WebDriverWait(navegador, 10).until(
            EC.url_contains("https://mantis.saojudas.base2.com.br/bug_report_page.php")        
        )
    finally:
        navegador.close()
        print("Criar tarefas em sequência - OK")
    
criar_tarefas_em_sequencia()

Criar tarefas em sequência - OK
